In [1]:
# trong env của bạn
# ! pip install webrtcvad


tính SNR (tỉ số tín hiệu/nhiễu) cho từng file WAV thì chuẩn nhất là:

1. cắt audio thành các khung ngắn (10–30 ms)
2. dùng VAD (voice activity detection) để gắn nhãn **speech** / **non‑speech** cho từng khung
3. tính năng lượng trung bình của khung **speech** = $P_\text{signal}$, của **non‑speech** = $P_\text{noise}$
4. $\text{SNR}_{\mathrm{dB}} = 10 \log_{10}\big((P_\text{signal}+\epsilon)/(P_\text{noise}+\epsilon)\big)$

Dưới đây là 1 script gọn bạn chạy ngay trong env hiện tại (dùng `torchaudio` + tùy chọn `webrtcvad` nếu có). Script cũng đọc/ghi **manifest** của bạn và gắn `snr` + `snr_bucket` cho từng sample.

---

### Cài gọn (tùy chọn VAD tốt hơn)

```bash
# trong env của bạn
pip install webrtcvad
```

---

### Script: `tools/calc_snr.py`


### Cách chạy (khớp pipeline của bạn)

Giả sử manifest hiện tại của bạn có trường `wav` (hoặc `audio_path`) và là JSON‑Lines:

```bash
python tools/calc_snr.py \
  --in_manifest ~/dataset/data/manifests/user_only_train.jsonl \
  --out_manifest ~/dataset/data/manifests/user_only_train.with_snr.jsonl \
  --audio_root ~/dataset/data/data/processed/8khz/user_only
```

* Sau khi chạy, mỗi dòng sample có thêm:

  ```json
  {"wav":".../xxx.wav", "text":"...", "utt_id":"...", "snr": 14.23, "snr_bucket":"mid"}
  ```
* Bạn có thể tách dev set theo bucket:

  * `clean`: SNR > 20 dB
  * `mid`: 12–20 dB
  * `noisy`: ≤ 12 dB

---

### Gợi ý kiểm tra nhanh chất lượng VAD

* Nếu nhiều file bị `snr_bucket="unknown"` → file quá ngắn hoặc VAD không ổn; thử `--no_webrtc` để dùng energy‑VAD.
* Với dữ liệu 8 kHz: script đã resample về 16 kHz; `webrtcvad` vẫn chạy được ở 8/16/32/48 kHz.

---

### Mẹo thực tế

* **Không có non‑speech dài** (cắt sát tiếng nói): noise khó ước lượng. Khi đó script fallback dùng percentile năng lượng thấp.
* Đừng so sánh SNR giữa các file có **khác gain** quá lớn mà không chuẩn hoá; tuy nhiên vì ta tính theo *tỉ lệ*, rescale tuyến tính thường không làm hỏng SNR.
* Bạn có thể log thêm `speech_ratio = (#speech_frames / #frames)` để lọc những file “toàn noise”.

---

Muốn mình **nhúng thẳng SNR vào pipeline multi‑process** của bạn (đọc/ghi manifest + ghi log per‑dataset) thì mình viết thêm patch theo đúng đường dẫn code hiện tại (`chunkformer_vpb/preprocess/...`).


In [2]:
#!/usr/bin/env python3
import argparse, json, math, sys
from pathlib import Path

import torch
import torchaudio
from torchaudio.functional import resample

try:
    import webrtcvad
    HAVE_WEBRTCVAD = True
except Exception:
    HAVE_WEBRTCVAD = False

# ---------- core utils ----------
def to_mono_16k(waveform, sr, target_sr=16000):
    if waveform.dim() == 2 and waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if sr != target_sr:
        waveform = resample(waveform, sr, target_sr)
    return waveform.squeeze(0), target_sr

def frame_wave(wav, sr, frame_ms=20):
    N = int(sr * frame_ms / 1000)
    total = (len(wav) // N) * N
    if total == 0:
        return torch.empty(0, N), N
    w = wav[:total].view(-1, N)
    return w, N

def energy(frames):
    # RMS^2 (power); add small epsilon later
    return (frames.float() ** 2).mean(dim=1)

def vad_mask_webrtc(frames, sr, frame_len, aggressiveness=2):
    # frames: [num_frames, frame_len] float32 in [-1,1]
    vad = webrtcvad.Vad(aggressiveness)
    # convert each frame to 16-bit PCM bytes
    frames_int16 = (frames.clamp(-1, 1) * 32768.0).short().numpy()
    import struct
    mask = []
    for fr in frames_int16:
        pcm = struct.pack("<%dh" % len(fr), *fr.tolist())
        is_speech = vad.is_speech(pcm, sr)
        mask.append(is_speech)
    return torch.tensor(mask, dtype=torch.bool)

def vad_mask_energy(frames, threshold_scale=0.5):
    # đơn giản: threshold = 0.5 * median energy
    en = energy(frames)
    thr = threshold_scale * en.median().item()
    return en > thr

def compute_snr_for_wav(wav_path, target_sr=16000, use_webrtc=True, frame_ms=20):
    wav, sr = torchaudio.load(wav_path)
    wav, sr = to_mono_16k(wav, sr, target_sr)
    if wav.numel() < sr * 0.2:
        return None  # quá ngắn

    frames, N = frame_wave(wav, sr, frame_ms)
    if frames.numel() == 0:
        return None

    if use_webrtc and HAVE_WEBRTCVAD and sr in (8000, 16000, 32000, 48000) and frame_ms in (10, 20, 30):
        speech_mask = vad_mask_webrtc(frames, sr, N, aggressiveness=2)
    else:
        speech_mask = vad_mask_energy(frames, threshold_scale=0.5)

    en = energy(frames) + 1e-12
    # Nếu không có đủ non-speech, ước lượng noise bằng percentile thấp
    if speech_mask.any():
        sig_power = en[speech_mask].mean().item()
    else:
        # fallback: lấy top 50% energy làm signal
        sig_power = en.quantile(0.75).item()

    if (~speech_mask).any():
        noise_power = en[~speech_mask].mean().item()
    else:
        # fallback: lấy 10% thấp nhất làm noise
        noise_power = en.quantile(0.10).item()

    snr_db = 10.0 * math.log10((sig_power) / (noise_power + 1e-12))
    return snr_db

def snr_bucket(snr_db):
    if snr_db is None: return "unknown"
    if snr_db > 20: return "clean"
    if snr_db >= 12: return "mid"
    return "noisy"

# ---------- CLI for manifest ----------# ---------- CLI for manifest (JSON array) ----------
import sys, json
from pathlib import Path
from collections import Counter

def process_manifest(in_manifest, out_manifest, audio_root=None, use_webrtc=True,
                     debug_every=0):
    """
    Đọc manifest dạng JSON array, tính SNR cho từng item, ghi ra JSON array mới.

    Args:
        in_manifest: đường dẫn file JSON (array of items)
        out_manifest: đường dẫn file JSON kết quả (array)
        audio_root: prepend vào audio_path nếu là relative
        use_webrtc: bật/tắt WebRTC VAD (nếu có)
        debug_every: >0 thì in log mỗi N sample (stderr)
    """
    audio_root = Path(audio_root) if audio_root else None

    # 1) Đọc JSON array
    with open(in_manifest, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except Exception as e:
            sys.stderr.write(f"[ERR] Not a valid JSON array: {e}\n")
            raise

    if not isinstance(data, list):
        raise ValueError("Input manifest must be a JSON array.")

    kept = []
    buckets = []

    # 2) Lặp và tính SNR
    for i, obj in enumerate(data):
        if not isinstance(obj, dict):
            continue

        wav_path = obj.get("wav") or obj.get("audio_path")
        if audio_root and wav_path and not Path(wav_path).is_absolute():
            wav_path = str(audio_root / wav_path)

        snr = None
        err_msg = None
        try:
            snr = compute_snr_for_wav(wav_path, use_webrtc=use_webrtc)
        except Exception as e:
            err_msg = str(e)

        if snr is None:
            if err_msg:
                sys.stderr.write(f"[WARN] SNR fail: {wav_path} -> {err_msg}\n")
        else:
            if debug_every and (i % debug_every == 0):
                sys.stderr.write(f"[SNR] {i:>6} | {snr:6.2f} dB | {wav_path}\n")

        obj["snr"] = None if snr is None else round(snr, 2)
        bucket = snr_bucket(snr)
        obj["snr_bucket"] = bucket
        buckets.append(bucket)
        kept.append(obj)

    # 3) Ghi JSON array đầu ra (pretty)
    with open(out_manifest, "w", encoding="utf-8") as f:
        json.dump(kept, f, ensure_ascii=False, indent=2)

    # 4) Thống kê nhanh
    c = Counter(buckets)
    total = len(buckets)
    dist = {k: f"{v} ({v/total:.1%})" for k, v in sorted(c.items(), key=lambda x: x[0])}
    sys.stderr.write(f"[OK] Wrote: {out_manifest}\n")
    sys.stderr.write(f"[STATS] total={total} | buckets={dist}\n")



/home/kylh/.local/share/mamba/envs/stt310/lib/python3.10/site-packages/webrtcvad.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# if __name__ == "__main__":
#     ap = argparse.ArgumentParser()
#     ap.add_argument("--in_manifest", required=True, help="manifest input (jsonlines)")
#     ap.add_argument("--out_manifest", required=True, help="manifest output (jsonlines)")
#     ap.add_argument("--audio_root", default=None, help="prepend to relative wav paths")
#     ap.add_argument("--no_webrtc", action="store_true", help="disable webrtcvad even if installed")
#     args = ap.parse_args()

# process_manifest(args.in_manifest, args.out_manifest, args.audio_root, use_webrtc=(not args.no_webrtc))

In [4]:
in_manifest = "../../../vpb_dataset/manifest_vpb_right/train_meta_right_2_filtered.json"
out_manifest = "../../../vpb_dataset/manifest_vpb_right/train_meta_right_2_filtered_snr.json"
audio_root = "../../../vpb_dataset/"

process_manifest(in_manifest, out_manifest, audio_root, use_webrtc=(not False))

[OK] Wrote: ../../../vpb_dataset/manifest_vpb_right/train_meta_right_2_filtered_snr.json
[STATS] total=3366 | buckets={'clean': '2433 (72.3%)', 'mid': '639 (19.0%)', 'noisy': '279 (8.3%)', 'unknown': '15 (0.4%)'}


In [5]:
in_manifest = "../../../vpb_dataset/manifest_vpb_right/valid_meta_filtered.json"
out_manifest = "../../../vpb_dataset/manifest_vpb_right/valid_meta_filtered_snr.json"
audio_root = "../../../vpb_dataset"

process_manifest(in_manifest, out_manifest, audio_root, use_webrtc=(not False))

[OK] Wrote: ../../../vpb_dataset/manifest_vpb_right/valid_meta_filtered_snr.json
[STATS] total=735 | buckets={'clean': '441 (60.0%)', 'mid': '189 (25.7%)', 'noisy': '100 (13.6%)', 'unknown': '5 (0.7%)'}
